In [13]:
# Obsolete
from config import setup
setup()

 # Projet AP GeoSafe, Evacuation lors de feu
 
 ## Generator 
 Pour générer les données utilisées après par le projet, il faut utiliser le générateur du projet evacsim

In [15]:
from docplex.cp.model import CpoModel
from docplex.cp.model import *
import numpy as np

configuration

In [16]:
from config_duc import setup
setup()

## modélisation

In [17]:
def geosafe_model(l, q, eps, r, W, H, d):
    '''       
    Int  Matrix l: length path from i to j = G.l[i,j]. No path ij <--> G.l[i,j] = 0
    Int  Array  q: capacity of node i <--> G.q[i]
    Bool Array  eps: evacuation node 
    Bool Array  r: safe node
    Int  Array  W: initial population at node i <--> W[i]
    Int         H : Time span
    Int  Array  d: deadline to leave the node
    '''
    
    nb_node = q.shape[0]
    nodes = np.arange(nb_node) # id nodes, i.e [1,2,3,4, ...]
    total_population = np.sum(W)
    
    t = !eps & !r # transition node

    mdl = CpoModel(name='geosafe')
    
    # == Output ==
    # starting date
    s = np.array( mdl.integer_var_list(nb_node, min=0, max = 150, name="s") )
    
    # evacuation rate aka. height of package
    h = np.array( mdl.integer_var_list(nb_node, min=0, max = 300, name="h") )

    # == Intermediate ==
    # node flow of population
    phi = np.matrix([
            [ 
                mdl.integer_var(name='phi[%d,%d]'%(i,j)) for j in range(H) 
            ] for i in range(nb_node)
        ])
    
    # ending date (leaving time) of node
    e = np.array( mdl.integer_var_list(nb_node, min=0, max = 150, name= "e") )
    
    for v in range(nb_node):
        if eps[v]:
            mdl.add(
                e[v] == s[v] + W[v]/h[v]
            )
    
    # == Constraints ==
    # evacuate everyone in evacuation node
    for v in range(nb_node):
        mdl.add(
            mdl.sum(phi[v, t] for t in range (H)) == W[v] # TODO not really understand this constraint
        ) 
    
    # Flow at a node u = sum from all of its leaves node epsilon
#     mdl.add_constraint(
#         phi[u, t] == np.sum( phi[u, t - l[u,v] - s[eps]] ) \
#             for t in range(H)                                  \
#             for v in np.where(eps)                           \
#             for u in np.where(t)                             
#     )

    # simplified version
    for u in nodes[np.where(t)]:
        for t in range(H):
             
            v = nodes[np.where(
                                np.logical_and(
                                    l[u,:] > 0, 
                                    s + l[u,:] <= t,
                                    t <= e + l[u, :]
                                )
                               )[1]]             
            mdl.add(
                phi[u, t] == mdl.sum(h[v])
            )
    
    # Flow at a node does not excess capacity of arc
    for t in range(H):
        for u in nodes:
            mdl.add(
                phi[u,t] <= q[u]
            )
    
    # Objective
    mdl.add(
        minimize(np.max(s[eps] + h[eps]/W[eps] - d[eps]))
    )
    
    return mdl

## Unit test:

- Test input

In [18]:
import networkx as nx

In [19]:
G = nx.Graph()

global_d = 250 # TODO: delete this one, just for debug

# TODO: Maybe we should use eps = W > 0, so that do not need to use 'eps' attribute?
nodes_state = [
    (1, {'eps': True,  'r': False, 'W': 5, 'd': global_d}),
    (2, {'eps': True,  'r': False, 'W': 5, 'd': global_d}),
    (3, {'eps': True,  'r': False, 'W': 5, 'd': global_d}),
    (4, {'eps': False, 'r': False, 'W': 0, 'd': global_d}),
    (5, {'eps': False, 'r': False, 'W': 0, 'd': global_d}),
    (6, {'eps': False, 'r': True,  'W': 0, 'd': global_d})
]

edges_state = [
    (1, 4, {'l' : 4}),
    (2, 4, {'l' : 3}),
    (4, 5, {'l' : 7}),
    (3, 5, {'l' : 3}),
    (5, 6, {'l' : 10})
]

G.add_nodes_from(nodes_state)
G.add_edges_from(edges_state)

In [20]:
def _node_attribute_(G, attribute='eps'):
    '''
    Author: Duc Hau :D
    Return a numpy boolean array of attribute
    '''
    values_tmp = nx.get_node_attributes(G, attribute).values()
    return np.array(list(values_tmp))

In [10]:
print(_node_attribute_(G, attribute='W'))

[5 5 5 0 0 0]


- Init model

In [21]:
# Path length
l = nx.to_numpy_matrix(G, weight='l')

# Node capacities (shouldn't it be the path??)
q = np.array([20,20,20,20,20,20])

# If the node is in EPSILON (evacuating node)
eps = _node_attribute_(G, 'eps')

# If the node is in the ROOT (safe node)
r = _node_attribute_(G, 'r')

# Initial population
W = _node_attribute_(G, 'W')

# Time span
H = 200

# Deadline for evacutation
d = _node_attribute_(G, 'd')

# Solver configuration
ctx = {}

In [22]:
mdl = geosafe_model(l, q, eps, r, W, H, d)

- solve model

In [27]:
mdl.solve()
sol = mdl.solve(Presolve='On', Workers='Auto')
print(sol.get_solver_log())

print('\n\n Result:')
if sol.is_solution():
    sol.print_solution()
else:
    print('Problem does not have solution')
    sol.print_solution()

 ! ----------------------------------------------------------------------------
 ! Minimization problem - 1210 variables, 1609 constraints
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 9643.2 (before), 9643.2 (after)
 !  . Memory usage      : 1.0 MB (before), 1.0 MB (after)
 ! Using parallel search with 12 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0       1210                 -
 + New bound is -199.801
 ! ----------------------------------------------------------------------------
 ! Search completed, model has no solution.
 ! Best bound             : -199.801
 ! Number of branches     : 2
 ! Number of fails        : 7
 ! Total memory usage     : 5.1 MB (5.0 MB CP Optimizer + 0.1 MB Concert)
 ! Time spent in solve    : 0.02s (0.02s engine + 0.00s extraction)
 ! Search speed (br. / s) : 100.0
 ! --------